In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix,precision_score,recall_score
from sklearn import preprocessing
import imblearn
from imblearn.over_sampling import RandomOverSampler

In [2]:
t1=pd.read_csv('transactions_1.csv')
t2=pd.read_csv('transactions_2.csv')
t3=pd.read_csv('transactions_3.csv')
t = pd.concat([t1, t2, t3])

users=pd.read_csv('users.csv')

users_new=users.drop(['country','city','user_settings_crypto_unlocked','attributes_notifications_marketing_push', 'attributes_notifications_marketing_email','num_contacts','num_referrals','num_successful_referrals'], axis=1)

In [3]:
a=[] 
b=[]
for i in users_new['created_date']:
    a.append(i[0:4])
    b.append(i[5:7])

users_new=users_new.assign(year = a,month = b)

In [4]:
users_n2=users_new.copy()
users_n2=users_n2.drop(['created_date'],axis=1)

In [5]:
c=[] 
for i in users_n2['user_id']:
    c.append(i[5:])

users_n2=users_n2.assign(id1= c)

In [6]:
users_n2=users_n2.drop(['user_id'],axis=1)

In [7]:
users_n2 = users_n2.rename({'id1': 'user_id'}, axis=1)

In [8]:
d=[] 
for i in users_n2['birth_year']:
    d.append(2020-int(i))
    
users_n3=users_n2.copy()
users_n3=users_n3.assign(age = d)
users_n4=users_n3.drop(['birth_year'],axis=1)

In [9]:
f=[] 
for i in t['user_id']:
    f.append(i[5:])

t=t.assign(id1= f)

In [10]:
t=t.drop(['user_id'],axis=1)
t = t.rename({'id1': 'user_id'}, axis=1)

In [11]:
t=t[t['transactions_state']=='COMPLETED']

In [12]:
u=[]
x=0
for i in t['user_id']:
    if x==0:
        u.append(i)
        x=x+1
    elif i not in u:
        u.append(i)
    else:continue

In [13]:
u1=[]
for i in u:
    i1=int(i)
    u1.append(i1)
u1=sorted(u1)

In [14]:
users_n4['user_id']=users_n4['user_id'].astype(int)
h=users_n4['user_id']

In [15]:
u_n2=[]
for i in h:
    if i not in u1:
        u_n2.append(i)
for i in u_n2:
    i=int(i)

In [16]:
users_n4=users_n4.set_index('user_id')

In [17]:
for i in u_n2:
    users_n4=users_n4.drop([i])

In [18]:
users_n5=users_n4.reset_index()

In [19]:
k=[] 
l=[]
for i in t['created_date']:
    k.append(i[0:4])
    l.append(i[5:7])

t=t.assign(year = k,month = l)
t=t.drop(['created_date'],axis=1)

In [20]:
t['user_id']=t['user_id'].astype(int)
pr=[]
for i in u1:
    df=t[t['user_id']==i]
    x=df['amount_usd'].sum()
    pr.append(x)

In [21]:
users_n5=users_n5.assign(monetary_value = pr)

In [22]:
t['user_id']=t['user_id'].astype(int)
fr=[]
x=0
for i in u1:
    df=t[t['user_id']==i]
    x=len(df['amount_usd'])
    fr.append(x)

In [23]:
users_n6=users_n5.assign(frequency_value = fr)

In [24]:
age_dict={'2018':0,'2019':12}
users_n6['year']=users_n6['year'].map(age_dict)

In [25]:
users_n6['year']=users_n6['year'].astype(int)
users_n6['month']=users_n6['month'].astype(int)

In [26]:
users_n7=users_n6.copy()
users_n7['active']=17-(users_n7['year']+users_n7['month'])

In [27]:
users_n7['monetary_value']=users_n7['monetary_value']/users_n7['active']
users_n7['frequency_value']=users_n7['frequency_value']/users_n7['active']

In [28]:
age_dict={'2018':0,'2019':12}
t['year']=t['year'].map(age_dict)
t['month']=t['month'].astype(int)
tt=t.copy()
tt['touches']=tt['year']+tt['month']

In [29]:
tt1=tt.sort_values(['user_id', 'year','month'])

In [30]:
x=users_n7['user_id']
x1=[]
for i in x:
    i1=int(i)
    x1.append(i1)
    
re=[]
for j in x1:
    df=tt1[tt1['user_id']==j]
    y=df.iloc[-1,-1]
    re.append(y)

In [31]:
users_n7=users_n7.assign(touches = re)

In [32]:
users_n7['creation']=17
users_n7['recency_value']=users_n7['creation']-users_n7['touches']

In [33]:
users_n7=users_n7.drop(['year','month','creation','touches'],axis=1)

In [34]:
users_n8=users_n7.drop(['active'],axis=1)

In [36]:
m={16:1,15:1,14:1,13:1,12:1,11:1,10:1,9:1,8:1,7:2,6:2,5:2,4:2,3:2,2:3,1:3,0:3}
users_n8["recency_value"]=users_n8["recency_value"].map(m)

In [37]:
n={'STANDARD':0, 'GOLD':1, 'SILVER':1}
users_n8["plan"]=users_n8["plan"].map(n)

In [39]:
users_n8.to_csv('Final_Table_mapping.csv', index=False)